# Import required libraries


In [1]:
import random
import re
from pprint import pprint

import pandas as pd
import requests
import stanza

# Import all sheets from the 7000 Sentences dataset


In [2]:
df_7000_sents_1 = pd.read_excel(
    "../data/corpus/7000 Sentences Corpus With IDs.xlsx",
    sheet_name="3000",
    usecols=["ID", "French", "Hindi"],
    na_values=['NA'],
)

df_7000_sents_2 = pd.read_excel(
    "../data/corpus/7000 Sentences Corpus With IDs.xlsx",
    sheet_name="6000",
    usecols=["ID", "French", "Hindi"],
    na_values=['NA'],
)

df_7000_sents_3 = pd.read_excel(
    "../data/corpus/7000 Sentences Corpus With IDs.xlsx",
    sheet_name="1000",
    usecols=["ID", "French", "Hindi"],
    na_values=['NA'],
)

# Import the 3000 Words dataset


In [3]:
df_3000_words = pd.read_excel(
    "../data/corpus/3000 Hindi Words Corpus With IDs.xlsx",
    sheet_name="Feuille1",
    usecols=["ID", "French Word", "Hindi Word"],
    na_values=["NA"],
)

# Download a Stanza Language Model for Hindi and French into the directory "../stanza_models"


In [4]:
stanza.download(lang='hi')

2022-10-13 16:21:12 INFO: Downloading default packages for language: hi (Hindi) ...
2022-10-13 16:21:13 INFO: File exists: C:\Users\Khushi\stanza_resources\hi\default.zip
2022-10-13 16:21:14 INFO: Finished downloading models and saved to C:\Users\Khushi\stanza_resources.


# Initialize a Stanza pipeline with a language model for Hindi and French

## Which is assigned to the variable 'nlp_hi' and 'nlp_fr' using the Pipeline() class


In [5]:
nlp_hi_stanza = stanza.Pipeline(lang='hi')

2022-10-13 16:21:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-13 16:21:15 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |
| depparse  | hdtb    |

2022-10-13 16:21:15 INFO: Use device: cpu
2022-10-13 16:21:15 INFO: Loading: tokenize
2022-10-13 16:21:15 INFO: Loading: pos
2022-10-13 16:21:16 INFO: Loading: lemma
2022-10-13 16:21:16 INFO: Loading: depparse
2022-10-13 16:21:16 INFO: Done loading processors!


# Set Exercise Dataset Columns

In [6]:
cols = [
    "Exo_type_id",
    "Exo_type",
    "Exo_objective",
    "Exo_focus",
    "Exo_id",
    "Source_format",
    "Target_format",
    "Source_sentence_id",
    "Source_word_id",
    "Source_lang",
    "Target_lang",
    "Full_sentence",
    "Instruction",
    "Sentence_w_blank",
    "Right_answer",
    "Dist_1",
    "Dist_2",
    "Dist_3",
    "Explanation",
    "Difficulty",
    "Remediation",
]

# Distractor Creation Functions


## Spelling Distractors


In [7]:
# exchanges two letters in a given word
def exchange_letter(a, b, text, list_of_distractors):
    if (a in text):
        list_of_distractors.append(text.replace(a, b))

    if (b in text):
        list_of_distractors.append(text.replace(b, a))

    return (list_of_distractors)

# Mistakes with similar sounding and looking vowels
def vowel_changer(word, list_of_distractors):
    if ('इ' in word.text or 'ई' in word.text):
        list_of_distractors = exchange_letter(
            'इ', 'ई', word.text, list_of_distractors)

    if ('ि' in word.text or 'ी' in word.text):
        list_of_distractors = exchange_letter(
            'ि', 'ी', word.text, list_of_distractors)

    if ('उ' in word.text or 'ऊ' in word.text):
        list_of_distractors = exchange_letter(
            'उ', 'ऊ', word.text, list_of_distractors)

    if ('ु' in word.text or 'ू' in word.text):
        list_of_distractors = exchange_letter(
            'ु', 'ू', word.text, list_of_distractors)

    if ('ए' in word.text or 'ऐ' in word.text):
        list_of_distractors = exchange_letter(
            'ए', 'ऐ', word.text, list_of_distractors)

    if ('े' in word.text or 'ै' in word.text):
        list_of_distractors = exchange_letter(
            'े', 'ै', word.text, list_of_distractors)

    if ('ओ' in word.text or 'औ' in word.text):
        list_of_distractors = exchange_letter(
            'ओ', 'औ', word.text, list_of_distractors)

    if ('ो' in word.text or 'ौ' in word.text):
        list_of_distractors = exchange_letter(
            'ो', 'ौ', word.text, list_of_distractors)

    if ('अं' in word.text or 'अँ' in word.text):
        list_of_distractors = exchange_letter(
            'अं', 'अँ', word.text, list_of_distractors)

    if ('ं' in word.text or 'ँ' in word.text):
        list_of_distractors = exchange_letter(
            'ं', 'ँ', word.text, list_of_distractors)

    if ('ॉ' in word.text or 'ाँ' in word.text):
        list_of_distractors = exchange_letter(
            'ॉ', 'ाँ', word.text, list_of_distractors)

    if ('ॉ' in word.text or 'ां' in word.text):
        list_of_distractors = exchange_letter(
            'ॉ', ' ां', word.text, list_of_distractors)

    if ('ाँ' in word.text or 'ां' in word.text):
        list_of_distractors = exchange_letter(
            'ाँ', 'ां', word.text, list_of_distractors)

    return list_of_distractors


# Mistakes with similar sounding consonents
def consonent_changer(word, list_of_distractors):
    if ('ट' in word.text or 'त' in word.text):
        list_of_distractors = exchange_letter(
            'ट', 'त', word.text, list_of_distractors)

    if ('ठ' in word.text or 'थ' in word.text):
        list_of_distractors = exchange_letter(
            'ठ', 'थ', word.text, list_of_distractors)

    if ('ड' in word.text or 'द' in word.text):
        list_of_distractors = exchange_letter(
            'ड', 'द', word.text, list_of_distractors)

    if ('ढ' in word.text or 'ध' in word.text):
        list_of_distractors = exchange_letter(
            'ढ', 'ध', word.text, list_of_distractors)

    if ('न' in word.text or 'ण' in word.text):
        list_of_distractors = exchange_letter(
            'न', 'ण', word.text, list_of_distractors)

    if ('श' in word.text or 'ष' in word.text):
        list_of_distractors = exchange_letter(
            'श', 'ष', word.text, list_of_distractors)

    if ('श' in word.text or 'स' in word.text):
        list_of_distractors = exchange_letter(
            'श', 'स', word.text, list_of_distractors)

    if ('स' in word.text or 'ष' in word.text):
        list_of_distractors = exchange_letter(
            'स', 'ष', word.text, list_of_distractors)

    return list_of_distractors

# Mistakes with similar looking letters
def letter_changer(word, list_of_distractors):
    if ('ज' in word.text or 'ज्ञ' in word.text):
        list_of_distractors = exchange_letter(
            'ज', 'ज्ञ', word.text, list_of_distractors)

    if ('क्ष' in word.text or 'श्र' in word.text):
        list_of_distractors = exchange_letter(
            'क्ष', 'श्र', word.text, list_of_distractors)

    return list_of_distractors

# Create 3 spelling distractors for a given word
def spelling_distractors(word, list_of_distractors=None):

    if list_of_distractors is None:
        list_of_distractors = []

    list_of_distractors = vowel_changer(word, list_of_distractors)
    if len(list_of_distractors) < 3:
        list_of_distractors = consonent_changer(word, list_of_distractors)
    
    if len(list_of_distractors) < 3:
        list_of_distractors = letter_changer(word, list_of_distractors)

    if len(list_of_distractors) >= 3:
        list_of_distractors = list_of_distractors[:3]
    else:
        list_of_distractors = []

    return list_of_distractors

# Creating Vocabulary Flashcards


In [8]:
def word_flashcards(df_source, exo_id = 0):
    data = []
    for i in range(len(df_source)):
        if df_source["Hindi Word"][i] != "NA":
            exo_id += 1
            data.append([
                "10",
                "Flashcards",
                "Learning vocabulary",
                "",
                str(exo_id),
                "text",
                "text",
                "",
                df_source["ID"][i],
                "French",
                "Hindi",
                str(df_source["Hindi Word"][i]),
                "",
                "",
                str(df_source["French Word"][i]),
                "",
                "",
                "",
                "",
                "",
                "",
            ])
    return exo_id, data

exo_id = 0

exo_id, data_w_fc_1 = word_flashcards(df_3000_words, exo_id)
df_w_fc_1 = pd.DataFrame(data_w_fc_1, columns=cols)

frames_w_fc = [df_w_fc_1]
df_w_fc = pd.concat(frames_w_fc)
df_w_fc

,Exo_type_id,Exo_type,Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,Source_sentence_id,Source_word_id,Source_lang,...,Full_sentence,Instruction,Sentence_w_blank,Right_answer,Dist_1,Dist_2,Dist_3,Explanation,Difficulty,Remediation
0,10,Flashcards,Learning vocabulary,,1,text,text,,1,French,...,nan,,,Beauté,,,,,,
1,10,Flashcards,Learning vocabulary,,2,text,text,,2,French,...,सच,,,Verité,,,,,,
2,10,Flashcards,Learning vocabulary,,3,text,text,,3,French,...,स्वतंत्रता/ स्वाधीनता,,,Liberté,,,,,,
3,10,Flashcards,Learning vocabulary,,4,text,text,,4,French,...,भाईचारा,,,Fraternité,,,,,,
4,10,Flashcards,Learning vocabulary,,5,text,text,,5,French,...,दया,,,Bonté,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,10,Flashcards,Learning vocabulary,,2996,text,text,,2996,French,...,nan,,,Plombier,,,,,,
2996,10,Flashcards,Learning vocabulary,,2997,text,text,,2997,French,...,nan,,,Garagiste,,,,,,
2997,10,Flashcards,Learning vocabulary,,2998,text,text,,2998,French,...,nan,,,Démissionner,,,,,,
2998,10,Flashcards,Learning vocabulary,,2999,text,text,,2999,French,...,nan,,,Informaticien,,,,,,


# Create Spelling MCQ Exercises


In [9]:
def spelling_mcq(df_source, exo_id, cols):
    data = []

    spellings = {col: [] for col in cols}


    for i in range(len(df_source)):
        doc_hi_phrase = nlp_hi_stanza(str(df_source["Hindi Word"][i]))

        for sent in doc_hi_phrase.sentences:
            for word in sent.words:
                list_of_distractors = []

                if word.text not in spellings["Right_answer"]:
                    list_of_distractors = spelling_distractors(word, list_of_distractors)

                    if list_of_distractors != []:
                        
                        exo_id += 1

                        spellings["Exo_type_id"].append("14")
                        spellings["Exo_type"].append("MCQ")
                        spellings["Exo_objective"].append("Learning vocabulary")
                        spellings["Exo_focus"].append("Spellings")
                        spellings["Exo_id"].append( str(exo_id))
                        spellings["Source_format"].append("text")
                        spellings["Target_format"].append("text")
                        spellings["Source_sentence_id"].append("")
                        spellings["Source_word_id"].append(str(df_source["ID"][i]))
                        spellings["Source_lang"].append("French")
                        spellings["Target_lang"].append("Hindi")
                        spellings["Full_sentence"].append(str(df_source["French Word"][i]))
                        spellings["Instruction"].append("Select the correct spelling:")
                        spellings["Sentence_w_blank"].append("")
                        spellings["Right_answer"].append(str(word.text))
                        spellings["Dist_1"].append(list_of_distractors[0]),
                        spellings["Dist_2"].append(list_of_distractors[1]),
                        spellings["Dist_3"].append(list_of_distractors[2]),
                        spellings["Explanation"].append("")
                        spellings["Difficulty"].append("")
                        spellings["Remediation"].append("")

    return exo_id, spellings

exo_id = 0

exo_id, data_spelling_mcq_1 = spelling_mcq(df_3000_words, exo_id, cols)
df_spelling_mcq_1 = pd.DataFrame(data_spelling_mcq_1, columns=cols)

frames_spelling_mcq = [df_spelling_mcq_1]
df_spelling_mcq = pd.concat(frames_spelling_mcq)
df_spelling_mcq

,Exo_type_id,Exo_type,Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,Source_sentence_id,Source_word_id,Source_lang,...,Full_sentence,Instruction,Sentence_w_blank,Right_answer,Dist_1,Dist_2,Dist_3,Explanation,Difficulty,Remediation
0,14,MCQ,Learning vocabulary,Spellings,1,text,text,,3,French,...,Liberté,Select the correct spelling:,,स्वतंत्रता/,स्वतँत्रता/,स्वटंट्रटा/,श्वतंत्रता/,,,
1,14,MCQ,Learning vocabulary,Spellings,2,text,text,,3,French,...,Liberté,Select the correct spelling:,,स्वाधीनता,स्वाधिनता,स्वाधीनटा,स्वाढीनता,,,
2,14,MCQ,Learning vocabulary,Spellings,3,text,text,,6,French,...,Méchanceté,Select the correct spelling:,,दुष्टता,दूष्टता,दुष्तता,दुष्टटा,,,
3,14,MCQ,Learning vocabulary,Spellings,4,text,text,,7,French,...,Elégance,Select the correct spelling:,,लालित्य/सौष्ठव,लालीत्य/सौष्ठव,लालित्य/सोष्ठव,लालिट्य/सौष्ठव,,,
4,14,MCQ,Learning vocabulary,Spellings,5,text,text,,9,French,...,Idéal,Select the correct spelling:,,आदर्श,आडर्श,आदर्ष,आदर्स,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,14,MCQ,Learning vocabulary,Spellings,603,text,text,,992,French,...,Infection,Select the correct spelling:,,संक्रमण,सँक्रमण,संक्रमन,शंक्रमण,,,
603,14,MCQ,Learning vocabulary,Spellings,604,text,text,,993,French,...,S'évanouir,Select the correct spelling:,,बेहोश,बैहोश,बेहौश,बेहोष,,,
604,14,MCQ,Learning vocabulary,Spellings,605,text,text,,994,French,...,Indigestion,Select the correct spelling:,,बदहज़मी,बदहज़मि,बडहज़मी,बदहज्ञ़मी,,,
605,14,MCQ,Learning vocabulary,Spellings,606,text,text,,995,French,...,Intoxication alimentaire,Select the correct spelling:,,फुड-पॉईज्निंग,फुड-पॉइज्निंग,फुड-पॉईज्नींग,फूड-पॉईज्निंग,,,


# Creating Useful Sentences Flashcards


In [10]:
def sentence_flashcards(df_source):
    data = []
    exo_id = 0
    for i in range(len(df_source)):
        if df_source["Hindi"][i] != "NaN":
            exo_id += 1
            data.append([
                "24",
                "Flashcards",
                "Useful sentences",
                "",
                str(exo_id),
                "text",
                "text",
                df_source["ID"][i],
                "",
                "French",
                "Hindi",
                str(df_source["Hindi"][i]),
                "",
                "",
                str(df_source["French"][i]),
                "",
                "",
                "",
                "",
                "",
                "",
            ])
    return data

data_s_fc_3 = sentence_flashcards(df_7000_sents_3)
df_s_fc_3 = pd.DataFrame(data_s_fc_3, columns=cols)

frames_s_fc = [df_s_fc_3]
df_s_fc = pd.concat(frames_s_fc)
df_s_fc

,Exo_type_id,Exo_type,Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,Source_sentence_id,Source_word_id,Source_lang,...,Full_sentence,Instruction,Sentence_w_blank,Right_answer,Dist_1,Dist_2,Dist_3,Explanation,Difficulty,Remediation
0,24,Flashcards,Useful sentences,,1,text,text,6001,,French,...,हां,,,Oui,,,,,,
1,24,Flashcards,Useful sentences,,2,text,text,6002,,French,...,नहीं,,,Non,,,,,,
2,24,Flashcards,Useful sentences,,3,text,text,6003,,French,...,कृपया,,,S'il-vous-plaît,,,,,,
3,24,Flashcards,Useful sentences,,4,text,text,6004,,French,...,"धन्यवाद, धन्यवाद",,,Merci,,,,,,
4,24,Flashcards,Useful sentences,,5,text,text,6005,,French,...,कोई बात नहीं.,,,Je vous/t' en prie,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,24,Flashcards,Useful sentences,,996,text,text,6996,,French,...,यह गर्म आज नहीं है!,,,Qu'est-ce-qu'il fait chaud aujourd'hui!,,,,,,
996,24,Flashcards,Useful sentences,,997,text,text,6997,,French,...,यह धूमिल कल होगा,,,Il y aura du brouillard demain,,,,,,
997,24,Flashcards,Useful sentences,,998,text,text,6998,,French,...,चोर बंद करो!,,,Au voleur !,,,,,,
998,24,Flashcards,Useful sentences,,999,text,text,6999,,French,...,मैं चोरी हो गया था,,,On m'a volé,,,,,,


# Creating Verb Conjugation MCQ Exercises


In [11]:
def verb_conjug_mcq(df_source, exo_id, cols):
    
    verb_conjug = {col: [] for col in cols}

    for i in range(len(df_source)):
        doc_hi_phrase = nlp_hi_stanza(str(df_source["Hindi"][i]))

        count = 0
        blank = []
        answer = []
        hint = []

        for sent in doc_hi_phrase.sentences:
            for word in sent.words:
                if word.upos in ["VERB", "AUX"] and word.text != word.lemma:
                    blank.append(count)
                    answer.append(word.text)
                    hint.append(word.lemma)

                count += 1

        if len(blank):
            question = []

            for b in range(len(blank)):

                question.append(
                    " ".join([
                        word.text 
                        if word.id != blank[b] + 1
                        else "..." 
                        for word in sent.words 
                        for sent in doc_hi_phrase.sentences
                    ])
                )

                list_of_distractors = []
                list_of_distractors = spelling_distractors(nlp_hi_stanza(answer[b]), list_of_distractors)

                if list_of_distractors != []:

                    exo_id += 1

                    verb_conjug["Exo_type_id"].append("35")
                    verb_conjug["Exo_type"].append("MCQ")
                    verb_conjug["Exo_objective"].append("Grammar")
                    verb_conjug["Exo_focus"].append(
                        "Verb Conjugation")
                    verb_conjug["Exo_id"].append(str(exo_id))
                    verb_conjug["Source_format"].append("text")
                    verb_conjug["Target_format"].append("text")
                    verb_conjug["Source_sentence_id"].append(
                        str(df_source["ID"][i]))
                    verb_conjug["Source_word_id"].append("")
                    verb_conjug["Source_lang"].append("French")
                    verb_conjug["Target_lang"].append("Hindi")
                    verb_conjug["Full_sentence"].append(
                        str(df_source["French"][i]))
                    verb_conjug["Instruction"].append(
                        "Conjugate the verb correctly:")
                    verb_conjug["Sentence_w_blank"].append(str(f"{question[b]} ({hint[b]})"))
                    verb_conjug["Right_answer"].append(str(answer[b]))
                    verb_conjug["Dist_1"].append(list_of_distractors[0]),
                    verb_conjug["Dist_2"].append(list_of_distractors[1]),
                    verb_conjug["Dist_3"].append(list_of_distractors[2]),
                    verb_conjug["Explanation"].append("")
                    verb_conjug["Difficulty"].append("")
                    verb_conjug["Remediation"].append("")

    return exo_id, verb_conjug

exo_id = 0

exo_id, data_v_conjug_mcq_1 = verb_conjug_mcq(df_7000_sents_1, exo_id, cols)
df_v_conjug_mcq_1 = pd.DataFrame(data_v_conjug_mcq_1, columns=cols)

exo_id, data_v_conjug_mcq_2 = verb_conjug_mcq(df_7000_sents_2, exo_id, cols)
df_v_conjug_mcq_2 = pd.DataFrame(data_v_conjug_mcq_2, columns=cols)

exo_id, data_v_conjug_mcq_3 = verb_conjug_mcq(df_7000_sents_3, exo_id, cols)
df_v_conjug_mcq_3 = pd.DataFrame(data_v_conjug_mcq_3, columns=cols)

frames_v_conjug_mcq = [df_v_conjug_mcq_1, df_v_conjug_mcq_2, df_v_conjug_mcq_3]
df_v_conjug_mcq = pd.concat(frames_v_conjug_mcq)
df_v_conjug_mcq

,Exo_type_id,Exo_type,Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,Source_sentence_id,Source_word_id,Source_lang,...,Full_sentence,Instruction,Sentence_w_blank,Right_answer,Dist_1,Dist_2,Dist_3,Explanation,Difficulty,Remediation
0,35,MCQ,Grammar,Verb Conjugation,1,text,text,3,,French,...,"Dans une dictature, la liberté d'expression es...",Conjugate the verb correctly:,तानाशाही मे अभिव्यक्ति की स्वतंत्रता/ स्वाधीनत...,होती,होति,हौती,होटी,,,
1,35,MCQ,Grammar,Verb Conjugation,2,text,text,15,,French,...,Sa mère fait de très bonnes tartes,Conjugate the verb correctly:,उसकी मा बहोत स्वादिष्ट पाय ... है (बना),बनाती,बनाति,बनाटी,बणाती,,,
2,35,MCQ,Grammar,Verb Conjugation,3,text,text,26,,French,...,"Le chat est trop haut, je n'arrive pas à l'att...",Conjugate the verb correctly:,"बिल्ली बहोत उँचाई पर है , मैं वहाँतक पहोच नही ...",सकता,सकटा,शकता,षकता,,,
3,35,MCQ,Grammar,Verb Conjugation,4,text,text,32,,French,...,Il s'habille toujours en noir,Conjugate the verb correctly:,वो हमेशा काले कपड़े ... है (पहेन),पहेनता,पहैनता,पहेनटा,पहेणता,,,
4,35,MCQ,Grammar,Verb Conjugation,5,text,text,41,,French,...,Il est gentil uniquement quand il a besoin de toi,Conjugate the verb correctly:,वो तभी कृपालु / दयालु होता है जब उसे तुम्हारी ...,होती,होति,हौती,होटी,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,35,MCQ,Grammar,Verb Conjugation,2103,text,text,6957,,French,...,Je pense qu'on devrait en rester là,Conjugate the verb correctly:,मुझे लगता है हम यहाँ बंद कर ... चाहिए लगता है ...,देना,दैना,डेना,देणा,,,
219,35,MCQ,Grammar,Verb Conjugation,2104,text,text,6966,,French,...,ça te dirait d'aller à un match de football?,Conjugate the verb correctly:,एक फुटबॉल मैच के लिए ... के बारे में क्या ? (जा),जाने,जानै,जाणे,ज्ञाने,,,
220,35,MCQ,Grammar,Verb Conjugation,2105,text,text,6970,,French,...,Est-ce que tu pourrais me rendre un service?,Conjugate the verb correctly:,आप मेरा एक काम कर ... हैं ? (सक),सकते,सकतै,सकटे,शकते,,,
221,35,MCQ,Grammar,Verb Conjugation,2106,text,text,6981,,French,...,Est-ce que je peux parler à mon avocat?,Conjugate the verb correctly:,मैं अपने वकील से बात कर ... हैं ? (सक),सकते,सकतै,सकटे,शकते,,,


# Creating Verb Conjugation Cloze Test Exercises


In [12]:
def verb_conjug_cloze_test(df_source, exo_id):
    data = []
    for i in range(len(df_source)):
        doc_hi_phrase = nlp_hi_stanza(str(df_source["Hindi"][i]))

        count = 0
        blank = []
        answer = []
        hint = []
        for sent in doc_hi_phrase.sentences:
            for word in sent.words:
                if word.upos in ["VERB", "AUX"] and word.text != word.lemma:
                    blank.append(count)
                    answer.append(word.text)
                    hint.append(word.lemma)

                count += 1

        if len(blank):
            question = []

            for b in range(len(blank)):
                exo_id += 1
                question.append(
                    " ".join([
                        word.text 
                        if word.id != blank[b] + 1 
                        else "..." 
                        for word in sent.words 
                        for sent in doc_hi_phrase.sentences
                    ])
                )
                data.append([
                    "38", 
                    "Cloze_Test", 
                    "Verb_Conjugation", 
                    "Grammar", 
                    str(exo_id), 
                    "text", 
                    "text", 
                    str(df_source["ID"][i]), 
                    "", 
                    "French", 
                    "Hindi", 
                    str(df_source["Hindi"][i]), 
                    "Conjugate the verb correctly:", 
                    f"{question[b]} ({hint[b]})", 
                    str(answer[b]), 
                    "", 
                    "", 
                    "", 
                    "", 
                    "", 
                    "",
                ])

    return exo_id, data

exo_id = 0

exo_id, data_v_conjug_ct_1 = verb_conjug_cloze_test(df_7000_sents_1, exo_id)
df_v_conjug_ct_1 = pd.DataFrame(data_v_conjug_ct_1, columns=cols)

exo_id, data_v_conjug_ct_2 = verb_conjug_cloze_test(df_7000_sents_2, exo_id)
df_v_conjug_ct_2 = pd.DataFrame(data_v_conjug_ct_2, columns=cols)

exo_id, data_v_conjug_ct_3 = verb_conjug_cloze_test(df_7000_sents_3, exo_id)
df_v_conjug_ct_3 = pd.DataFrame(data_v_conjug_ct_3, columns=cols)

frames_v_conjug_ct = [df_v_conjug_ct_1, df_v_conjug_ct_2, df_v_conjug_ct_3]
df_v_conjug_ct = pd.concat(frames_v_conjug_ct)
df_v_conjug_ct

,Exo_type_id,Exo_type,Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,Source_sentence_id,Source_word_id,Source_lang,...,Full_sentence,Instruction,Sentence_w_blank,Right_answer,Dist_1,Dist_2,Dist_3,Explanation,Difficulty,Remediation
0,38,Cloze_Test,Verb_Conjugation,Grammar,1,text,text,3,,French,...,तानाशाही मे अभिव्यक्ति की स्वतंत्रता/ स्वाधीनत...,Conjugate the verb correctly:,तानाशाही मे अभिव्यक्ति की स्वतंत्रता/ स्वाधीनत...,होती,,,,,,
1,38,Cloze_Test,Verb_Conjugation,Grammar,2,text,text,4,,French,...,"स्वतंत्रता, समता और बंधुभाव ये फ्रेंच रिपूब्ली...",Conjugate the verb correctly:,"स्वतंत्रता , समता और बंधुभाव ये फ्रेंच रिपूब्ल...",थे,,,,,,
2,38,Cloze_Test,Verb_Conjugation,Grammar,3,text,text,5,,French,...,उसने आपको दया की भावना से मदद नही की,Conjugate the verb correctly:,उसने आपको दया की भावना से मदद नही ... (कर),की,,,,,,
3,38,Cloze_Test,Verb_Conjugation,Grammar,4,text,text,6,,French,...,उसकी दुष्टता की कोई सीमा नही थी,Conjugate the verb correctly:,उसकी दुष्टता की कोई सीमा नही ... (था),थी,,,,,,
4,38,Cloze_Test,Verb_Conjugation,Grammar,5,text,text,7,,French,...,उसके लालित्य और सौष्ठव से सभा प्रभावित हो गयी,Conjugate the verb correctly:,उसके लालित्य और सौष्ठव से सभा प्रभावित हो ... ...,गयी,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,38,Cloze_Test,Verb_Conjugation,Grammar,9561,text,text,6995,,French,...,इंटरनेट का काम नहीं करता,Conjugate the verb correctly:,इंटरनेट का काम नहीं ... (कर),करता,,,,,,
1072,38,Cloze_Test,Verb_Conjugation,Grammar,9562,text,text,6997,,French,...,यह धूमिल कल होगा,Conjugate the verb correctly:,यह धूमिल कल ... (हो),होगा,,,,,,
1073,38,Cloze_Test,Verb_Conjugation,Grammar,9563,text,text,6999,,French,...,मैं चोरी हो गया था,Conjugate the verb correctly:,मैं चोरी हो ... था (जा),गया,,,,,,
1074,38,Cloze_Test,Verb_Conjugation,Grammar,9564,text,text,7000,,French,...,आप तोड़ रहे हैं,Conjugate the verb correctly:,आप तोड़ ... हैं (रह),रहे,,,,,,


# Merge All Exercise Dataframes


In [13]:
frames_hi_exercises = [df_w_fc, df_spelling_mcq, df_s_fc, df_v_conjug_mcq, df_v_conjug_ct]
df_hi_exercises = pd.concat(frames_hi_exercises)
df_hi_exercises

,Exo_type_id,Exo_type,Exo_objective,Exo_focus,Exo_id,Source_format,Target_format,Source_sentence_id,Source_word_id,Source_lang,...,Full_sentence,Instruction,Sentence_w_blank,Right_answer,Dist_1,Dist_2,Dist_3,Explanation,Difficulty,Remediation
0,10,Flashcards,Learning vocabulary,,1,text,text,,1,French,...,nan,,,Beauté,,,,,,
1,10,Flashcards,Learning vocabulary,,2,text,text,,2,French,...,सच,,,Verité,,,,,,
2,10,Flashcards,Learning vocabulary,,3,text,text,,3,French,...,स्वतंत्रता/ स्वाधीनता,,,Liberté,,,,,,
3,10,Flashcards,Learning vocabulary,,4,text,text,,4,French,...,भाईचारा,,,Fraternité,,,,,,
4,10,Flashcards,Learning vocabulary,,5,text,text,,5,French,...,दया,,,Bonté,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,38,Cloze_Test,Verb_Conjugation,Grammar,9561,text,text,6995,,French,...,इंटरनेट का काम नहीं करता,Conjugate the verb correctly:,इंटरनेट का काम नहीं ... (कर),करता,,,,,,
1072,38,Cloze_Test,Verb_Conjugation,Grammar,9562,text,text,6997,,French,...,यह धूमिल कल होगा,Conjugate the verb correctly:,यह धूमिल कल ... (हो),होगा,,,,,,
1073,38,Cloze_Test,Verb_Conjugation,Grammar,9563,text,text,6999,,French,...,मैं चोरी हो गया था,Conjugate the verb correctly:,मैं चोरी हो ... था (जा),गया,,,,,,
1074,38,Cloze_Test,Verb_Conjugation,Grammar,9564,text,text,7000,,French,...,आप तोड़ रहे हैं,Conjugate the verb correctly:,आप तोड़ ... हैं (रह),रहे,,,,,,


# Extract Exercise Dataframe to an Excel File


In [14]:
df_hi_exercises.to_excel("Hindi_Exercises.xlsx", index=False)